In [38]:
import numpy as np


def composite(ifrom, ito, values, comp_length=1.0, min_comp_length=-1.0):
    """
    cfrom,cto,clen,cvar,cacum= composite(ifrom, ito, values,
                                        comp_length = 1, min_comp_length=-1)

    Composite intervals in a single drillhole. The From-To
    intervals may be sorted.

    Parameters
    ----------
    ifrom, ito:     1D arrays of floats
        From - To  interval
    values :   1D array of floats/integers
        variable,
    comp_length: Optional, float, default 1.
        length of the compositing intervals
    min_comp_length: Optional, float, defaul -1.
        minimum length of the composite, if <=0 then min_comp_length = comp_length/2.

    Return
    -------
    (cfrom, cto, clen, cvar, cacum)
    cfrom, cto:  1D arrays of floats
         From, To composited intervals
    clen, cvar, cacum:  1D arrays of floats
         total length of intervals composited
         variable composited
         variable accumulated

    """

    assert ifrom.shape == ito.shape, "Error: ifrom and ito with different shape"
    assert all(ifrom < ito), "Error: ifrom >= ito, wrong or zero length intervals"
    assert all(np.isfinite(ifrom)), "Error: ifrom with not finite elements"
    assert all(np.isfinite(ito)), "Error: ito with not finite elements"

    if min_comp_length <= 0:
        min_comp_length = comp_length / 2.0

    ncomp = int(ito[-1] / comp_length + 1)
    nintrb = len(ifrom)

    # create the composite arrays
    cfrom = np.arange(0.0, ito[-1] + comp_length, comp_length)
    cto = cfrom + comp_length
    clen = np.zeros(cto.shape)
    cvar = np.zeros(cto.shape)
    cvar[:] = np.nan
    cacum = np.zeros(cto.shape)

    iprop = np.zeros(ito.shape)

    # for each composite
    for i in range(ncomp):

        # initialize proportions
        iprop[:] = 0

        # for each interval
        for l in range(nintrb):

            # ignore interval if variable is nan
            if np.isnan(values[l]):
                continue

            # case a, below the composite
            if ifrom[l] >= cto[i]:
                break

            # case b, over the composite
            if ito[l] <= cfrom[i]:
                continue

            # --these are overlap--

            # case overlap top or contained
            if ito[l] > cfrom[i] and ito[l] <= cto[i]:

                # case c, interval in composite
                if ifrom[l] >= cfrom[i]:
                    iprop[l] = ito[l] - ifrom[l]

                # case d, overlap top
                else:
                    iprop[l] = ito[l] - cfrom[i]

            # case e, composite in interval
            if ifrom[l] < cfrom[i] and ito[l] > cto[i]:
                iprop[l] = cto[i] - cfrom[i]
                continue

            # case f, overlap bottom
            if ifrom[l] >= cfrom[i] and ifrom[l] < cto[i] and ito[l] > cto[i]:
                iprop[l] = cto[i] - ifrom[l]
                continue

        clen[i] = np.nansum(iprop)

        if clen[i] > min_comp_length:
            cacum[i] = np.nansum(values * iprop)
            cvar[i] = cacum[i] / clen[i]  # wighted average
        else:
            cvar[i] = np.nan
            cacum[i] = np.nan

    return pd.DataFrame({"from":cfrom, "to":cto, "len":clen, "value":cvar}

In [39]:
import pandas as pd

In [40]:
assay = pd.read_csv("../data/assay.csv")

In [41]:
assay.head()

,DHID,FROM,TO,_len,DUM,_acum,azmm,dipm,xm,ym,...,azme,dipe,xe,ye,ze,dist_hw,dist_fw,D1_surf,D1_solid,Au
0,0,0.0,1.0,1.0,0.0,0.0,90.0,86.774025,4.561190,-1.229899e-09,...,90.0,86.774147,4.589327,-2.459786e-09,98.728762,-1.0,-1.0,-1.0,0,0.0
1,0,1.0,2.0,1.0,0.0,0.0,90.0,86.774147,4.617463,-3.689650e-09,...,90.0,86.774086,4.645599,-4.919526e-09,97.730346,-1.0,-1.0,-1.0,0,0.0
2,0,2.0,3.0,1.0,0.0,0.0,90.0,86.774086,4.673736,-6.149413e-09,...,90.0,86.774025,4.701872,-7.379312e-09,96.731931,-1.0,-1.0,-1.0,0,0.0
3,0,3.0,4.0,1.0,0.0,0.0,90.0,86.774086,4.730009,-8.609211e-09,...,90.0,86.774086,4.758146,-9.839098e-09,95.733516,-1.0,-1.0,-1.0,0,0.0
4,0,4.0,5.0,1.0,0.0,0.0,90.0,86.774086,4.786282,-1.106898e-08,...,90.0,86.774086,4.814419,-1.229887e-08,94.735100,-1.0,-1.0,-1.0,0,0.0


In [42]:
dh = assay[assay.DHID==1]

In [43]:
composite(dh.FROM.values, dh.TO.values, dh.Au.values, comp_length=5)

,from,to,len,value,cumulative
0,0.0,5.0,5.0,0.000000,0.000000
1,5.0,10.0,5.0,0.000000,0.000000
2,10.0,15.0,5.0,0.000000,0.000000
3,15.0,20.0,5.0,0.000000,0.000000
4,20.0,25.0,5.0,0.000000,0.000000
5,25.0,30.0,5.0,0.000000,0.000000
6,30.0,35.0,5.0,0.000000,0.000000
7,35.0,40.0,5.0,0.000000,0.000000
8,40.0,45.0,5.0,0.000000,0.000000
9,45.0,50.0,5.0,0.000000,0.000000


In [44]:
def composite_dh(dh, comp_len, var_name):
    comps =  composite(dh.FROM.values, dh.TO.values, dh[var_name].values, comp_len)
    return comps.reset_index()
    

In [45]:
groups = assay.groupby("DHID")
result = groups.apply(composite_dh, 5,"Au")

In [46]:
result.values

array([[ 0.,  5.,  5.,  0.,  0.],
       [ 5., 10.,  5.,  0.,  0.],
       [10., 15.,  5.,  0.,  0.],
       ...,
       [20., 25.,  5.,  0.,  0.],
       [25., 30.,  5.,  0.,  0.],
       [30., 35.,  0., nan, nan]])

In [50]:
comps = result.reset_index()

In [53]:
comps[comps.DHID==6]

,DHID,level_1,from,to,len,value,cumulative
102,6,0,0.0,5.0,5.0,0.000000,0.000000
103,6,1,5.0,10.0,5.0,0.000000,0.000000
104,6,2,10.0,15.0,5.0,0.000000,0.000000
105,6,3,15.0,20.0,5.0,0.000000,0.000000
106,6,4,20.0,25.0,5.0,0.000000,0.000000
107,6,5,25.0,30.0,5.0,0.000000,0.000000
108,6,6,30.0,35.0,5.0,0.000000,0.000000
109,6,7,35.0,40.0,5.0,0.000000,0.000000
110,6,8,40.0,45.0,5.0,0.000000,0.000000
111,6,9,45.0,50.0,5.0,0.000000,0.000000
